In [ ]:
import os
import sys
import time
import random
import string
import schedule
from PyQt6 import QtCore, QtGui, QtWidgets
from functools import partial
from pack import packZipFile,packTarFile
from unpack import unpackZipFile,unpackTarFile
from huffman import huffmanCompress,huffmanDecompress
from Lzma import lzmaCompress,lzmaDecompress
from encrypt import encrypt3DES,encryptAES
from decrypt import decrypt3DES,decryptAES
from backup import backupLocal,backupGitHub
from threading import Thread,Event
from settings import SettingsDialog
from config_manager import ConfigManager

#密钥生成，随机生成以大小写字母及数字组成的定长字符串作为密钥
def randomString(length):
    content=string.ascii_letters+string.digits
    return ''.join(random.choice(content) for _ in range(length))

#GUI界面设计
class Ui_Dialog(object):
    #GUI界面初始化
    def init(self,Dialog):
        #背景设计
        self.Dialog=Dialog
        Dialog.setObjectName("Dialog")
        Dialog.resize(528, 535)
        
        #提示框设计
        self.label = QtWidgets.QLabel(parent=Dialog)
        self.label.setGeometry(QtCore.QRect(190, 30, 300, 35))
        self.label.setObjectName("label")
        
        #选项框架设计
        self.verticalLayoutWidget = QtWidgets.QWidget(parent=Dialog)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(60, 70, 401, 431))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")

        '''#自动备份子进程启动
        self.backup_local_thread=backup_local_thread
        self.backup_online_thread=backup_online_thread'''

        #进入主界面
        self.setupUI(Dialog)

    #主界面设计
    def setupUI(self,Dialog):
        #清屏
        self.clear()

        # 初始化配置管理器
        self.config_manager = ConfigManager()
        
        #查询参数初始化，设置页面大小
        self.size = self.config_manager.get("system", "page_size")
    
        # 从配置加载初始目录
        self.path = self.config_manager.get("system", "initial_directory")

        #操作选项设计
        self.pushButton = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton.setObjectName("pushButton")
        self.verticalLayout.addWidget(self.pushButton)
        self.pushButton_2 = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_2.setObjectName("pushButton_2")
        self.verticalLayout.addWidget(self.pushButton_2)
        self.pushButton_3 = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_3.setObjectName("pushButton_3")
        self.verticalLayout.addWidget(self.pushButton_3)
        self.pushButton_4 = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_4.setObjectName("pushButton_4")
        self.verticalLayout.addWidget(self.pushButton_4)
        self.pushButton_5 = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_5.setObjectName("pushButton_5")
        self.verticalLayout.addWidget(self.pushButton_5)
        
        self.retranslateUi(Dialog)
        self.pushButton.clicked.connect(self.backupInit) # type: ignore
        self.pushButton_2.clicked.connect(self.restoreInit) # type: ignore
        self.pushButton_3.clicked.connect(self.openSettings) # type: ignore
        self.pushButton_4.clicked.connect(self.exitApplication) # type: ignore
        self.pushButton_5.clicked.connect(self.showAbout) # type: ignore
        QtCore.QMetaObject.connectSlotsByName(Dialog)

    def reset(self):
        #self.clear()
        self.setupUI(self.Dialog)
    
    #备份源路径选择初始化
    def backupInit(self):
        self.tag=0
        self.searchDir()

    #还原源路径选择初始化
    def restoreInit(self):
        self.tag=1
        self.searchDir()

    #备份目标路径选择初始化
    def backupDstInit(self):
        self.tag=2
        self.dst_path=self.path
        self.searchDst()

    #还原目标路径选择初始化
    def restoreDstInit(self):
        self.tag=3
        self.dst_path=self.path
        self.searchDst()
        
    #清屏操作，删除选项框架中的所有组件
    def clear(self):
        while self.verticalLayout.count():
            child = self.verticalLayout.takeAt(0)
            if child.widget():
                child.widget().deleteLater()

    #进入下级目录
    def Forward(self,file):
        if self.tag<2:
            self.path=f'{self.path}{file}/'
            self.searchDir()
        else:
            self.dst_path=f'{self.dst_path}{file}/'
            self.searchDst()

    #回到上级目录
    def Back(self):
        if self.tag<2:
            self.path='/'.join(self.path[:-1].split('/')[:-1])+'/'
            self.searchDir()
        else:
            self.dst_path='/'.join(self.dst_path[:-1].split('/')[:-1])+'/'
            self.searchDst()

    #文件展示，用于在备份/还原时展示并选择当前路径下可供备份/还原的文件
    def showFiles(self,page):
        self.clear()
        length=min(len(self.files)-page*self.size,self.size)
        _translate = QtCore.QCoreApplication.translate
        
        #选项设计，为当前路径下每个文件创建一个按钮
        for i in range(0,length):
            file_button = QtWidgets.QCheckBox(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", self.files[page*self.size+i]))
            if self.files[page*self.size+i] in self.targets:
                file_button.setChecked(True)
        
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.targetsChange, self.files[page*self.size+i]))
            self.verticalLayout.addWidget(file_button)
            
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))
            
        if page>0:
            self.last=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.last.setObjectName('last')
            self.last.setText(_translate('Dialog','上一页'))
            self.last.clicked.connect(partial(self.showFiles, page-1))
            self.verticalLayout.addWidget(self.last)

        if page*self.size+self.size<len(self.files):
            self.next=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.next.setObjectName('next')
            self.next.setText(_translate('Dialog','下一页'))
            self.next.clicked.connect(partial(self.showFiles, page+1))
            self.verticalLayout.addWidget(self.next)

        #确定按钮设计，表示已完成目标文件的选择
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        if self.tag==0:
            self.confirm.clicked.connect(self.backupDstInit)
        elif self.tag==1:
            self.confirm.clicked.connect(self.restoreDstInit)
        self.verticalLayout.addWidget(self.confirm)

        #取消按钮设计，取消当前操作，返回初始界面
        self.cancel=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.cancel.setObjectName('cancel')
        self.cancel.setText(_translate('Dialog','取消'))
        self.cancel.clicked.connect(self.reset)
        self.verticalLayout.addWidget(self.cancel)

    #源路径展示，用于在备份/还原时展示并选择源路径
    def showDirs(self,page):
        self.clear()
        
        #提示信息更新
        _translate = QtCore.QCoreApplication.translate
        self.label.setText(_translate("Dialog", f"请移动到目标文件所在目录\n当前路径为{self.path}"))
        
        length=min(len(self.files)-page*self.size,self.size)
        _translate = QtCore.QCoreApplication.translate
        #选项设计，为当前路径下每个目录创建一个按钮
        for i in range(0,length):
            file_button = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", self.files[page*self.size+i]))
            
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.Forward, self.files[page*self.size+i]))
            self.verticalLayout.addWidget(file_button)
            
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))
            
        if page>0:
            self.last=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.last.setObjectName('last')
            self.last.setText(_translate('Dialog','上一页'))
            self.last.clicked.connect(partial(self.showDirs, page-1))
            self.verticalLayout.addWidget(self.last)

        if page*self.size+self.size<len(self.files):
            self.next=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.next.setObjectName('next')
            self.next.setText(_translate('Dialog','下一页'))
            self.next.clicked.connect(partial(self.showDirs, page+1))
            self.verticalLayout.addWidget(self.next)
            
        #选项设计，回到上级目录
        if os.path.abspath(self.path)!='C:/':
            self.back=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.back.setObjectName('back')
            self.back.setText(_translate('Dialog','返回上级目录'))
            self.back.clicked.connect(self.Back)
            self.verticalLayout.addWidget(self.back)

        #确定按钮设计，表示已到达目标路径
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        if self.tag==0:
            self.confirm.clicked.connect(self.Pack)
        elif self.tag==1:
            self.confirm.clicked.connect(self.Decrypt)
        self.verticalLayout.addWidget(self.confirm)

        #取消按钮设计，取消当前操作，返回初始界面
        self.cancel=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.cancel.setObjectName('cancel')
        self.cancel.setText(_translate('Dialog','取消'))
        self.cancel.clicked.connect(self.reset)
        self.verticalLayout.addWidget(self.cancel)

    #目标路径展示，用于在备份/还原时展示并选择目标路径
    def showDstDirs(self,page):
        self.clear()
        
        #提示信息更新
        _translate = QtCore.QCoreApplication.translate
        self.label.setText(_translate("Dialog", f"请移动到目标路径\n当前路径为{self.path}"))
        
        length=min(len(self.files)-page*self.size,self.size)
        _translate = QtCore.QCoreApplication.translate
        #选项设计，为当前路径下每个目录创建一个按钮
        for i in range(0,length):
            file_button = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", self.files[page*self.size+i]))
            
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.Forward, self.files[page*self.size+i]))
            self.verticalLayout.addWidget(file_button)
            
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))

        if page>0:
            self.last=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.last.setObjectName('last')
            self.last.setText(_translate('Dialog','上一页'))
            self.last.clicked.connect(partial(self.showDstDirs, page-1))
            self.verticalLayout.addWidget(self.last)

        if page*self.size+self.size<len(self.files):
            self.next=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.next.setObjectName('next')
            self.next.setText(_translate('Dialog','下一页'))
            self.next.clicked.connect(partial(self.showDstDirs, page+1))
            self.verticalLayout.addWidget(self.next)
            
        #选项设计，回到上级目录
        if os.path.abspath(self.path)!='C:/':
            self.back=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.back.setObjectName('back')
            self.back.setText(_translate('Dialog','返回上级目录'))
            self.back.clicked.connect(self.Back)
            self.verticalLayout.addWidget(self.back)

        #确定按钮设计，表示已到达目标路径
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        if self.tag==2:
            self.confirm.clicked.connect(self.PackFile)
        elif self.tag==3:
            self.confirm.clicked.connect(self.DecryptFile)
        self.verticalLayout.addWidget(self.confirm)

        #取消按钮设计，取消当前操作，返回初始界面
        self.cancel=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.cancel.setObjectName('cancel')
        self.cancel.setText(_translate('Dialog','取消'))
        self.cancel.clicked.connect(self.reset)
        self.verticalLayout.addWidget(self.cancel)
    
    #源路径查询，移动到源文件所在路径
    def searchDir(self):
        self.files=[file for file in os.listdir(self.path) if os.path.isdir(f'{self.path}{file}')]
        self.showDirs(0)

    #目标路径查询，移动到终点路径
    def searchDst(self):
        self.files=[file for file in os.listdir(self.dst_path) if os.path.isdir(f'{self.dst_path}{file}')]
        self.showDstDirs(0)

    #zip打包，将若干个源文件打包成一个zip压缩包
    def PackZipFile(self):
        file_name, ok = QtWidgets.QInputDialog.getText(
            self.Dialog, 
            '输入压缩包名称', 
            '请输入压缩包文件名:', 
            text=self.targets[0]+'.zip'
        )
        if not ok or not file_name.strip():
            file_name=self.targets[0]+'.zip'
        self.file=file_name
        
        try:
            packZipFile(self.path,self.targets,self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败','操作失败',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件打包至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Compress()

    #rar打包，将所选文件打包成一个rar压缩包，后因rarfile库不支持操作而放弃
    '''
    def CompressRarFile(self):
        #compressZipFile(self.targets,self.targets[0]+'.rar')
        try:
            compressRarFile(self.targets,self.targets[0]+'.rar')
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败','操作失败',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功','已将所选文件压缩至'+self.targets[0]+'.rar',QtWidgets.QMessageBox.StandardButton.Ok)
        self.clear()
        self.setupUI(self.Dialog)
    '''

    #tgz打包，将若干个源文件打包成一个tgz压缩包
    def PackTarFile(self):
        file_name, ok = QtWidgets.QInputDialog.getText(
            self.Dialog, 
            '输入压缩包名称', 
            '请输入压缩包文件名:', 
            text=self.targets[0]+'.tgz'
        )
        if not ok or not file_name.strip():
            file_name=self.targets[0]+'.tgz'
        self.file=file_name
        try:
            packTarFile(self.path,self.targets,self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败','操作失败',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件打包至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Compress()

    #打包操作界面，选择打包方式
    def PackFile(self):
        self.clear()
        if not self.targets:
            QtWidgets.QMessageBox.critical(None,'操作失败','请选择至少一个文件进行备份',QtWidgets.QMessageBox.StandardButton.Ok)
            self.setupUI(self.Dialog)
        else:
            _translate = QtCore.QCoreApplication.translate

            #提示信息更新
            self.label.setText(_translate("Dialog", "请选择打包方式："))

            #操作选项设计
            self.pushButton= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.pushButton.setObjectName('pushButton')
            self.pushButton.setText(_translate("Dialog", 'zip打包'))
            self.pushButton.clicked.connect(self.PackZipFile)
            self.verticalLayout.addWidget(self.pushButton)
            self.pushButton_2= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.pushButton_2.setObjectName('pushButton_2')
            self.pushButton_2.setText(_translate("Dialog", 'tgz打包'))
            self.pushButton_2.clicked.connect(self.PackTarFile)
            self.verticalLayout.addWidget(self.pushButton_2)
            
            #取消按钮设计，取消当前操作，返回初始界面
            self.cancel=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.cancel.setObjectName('cancel')
            self.cancel.setText(_translate('Dialog','取消'))
            self.cancel.clicked.connect(self.reset)
            self.verticalLayout.addWidget(self.cancel)

    #zip解包，将一个zip压缩包解包为源文件
    def UnpackZipFile(self,file):
        try:
            unpackZipFile(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解包',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将{self.path}{file}解包',QtWidgets.QMessageBox.StandardButton.Ok)

    #rar解包，将一个rar压缩包解包为源文件，后因rarfile库不支持操作而放弃
    '''
    def ExtractRarFile(self,file):
        try:
            extractRarFile(file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{file}无法解压',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解压至{file[:-4]}',QtWidgets.QMessageBox.StandardButton.Ok)
    '''

    #tgz解包，将一个tgz压缩包解包为源文件
    def UnpackTgzFile(self,file):
        try:
            unpackTarFile(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解包',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将{self.path}{file}解包',QtWidgets.QMessageBox.StandardButton.Ok)

    #解包操作界面，根据文件后缀名识别其打包方式并自动执行相应解包操作
    def Unpack(self):
        self.clear()
        for file in self.targets:
            tag=file[-4:]
            if tag=='.zip':
                self.UnpackZipFile(file)
            elif tag=='.tgz':
                self.UnpackTgzFile(file)
        self.reset()

    #目标文件更改，根据文件勾选状态决定将其添加到或移出目标列表
    def targetsChange(self,file_name):
        if file_name in self.targets:
            self.targets.remove(file_name)
        else:
            self.targets.append(file_name)

    #备份操作界面，选择目标路径下需要备份的文件
    def Pack(self):
        self.clear()
        self.files=[file for file in os.listdir(self.path)]
        self.targets=[]

        #提示信息更新
        _translate = QtCore.QCoreApplication.translate
        self.label.setText(_translate("Dialog", "请选择需要备份的文件："))

        #文件选项设计，为目标路径下每个文件创建一个多选框类型的按钮
        self.showFiles(0)

    #霍夫曼压缩，将目标文件以霍夫曼编码方式进行压缩并添加相应前缀名
    def HuffmanCompress(self):
        try:
            huffmanCompress(self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法压缩',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            self.file=f'huffman_{self.file}'
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件压缩至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Encrypt()

    #LZMA压缩，将目标文件以LZMA方式进行压缩并添加相应前缀名
    def LZMACompress(self):
        try:
            lzmaCompress(self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法压缩',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            self.file=f'LZMA_{self.file}'
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件压缩至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Encrypt()

    #压缩操作界面，选择压缩方式
    def Compress(self):
        self.clear()
        _translate = QtCore.QCoreApplication.translate

        #提示信息更新
        self.label.setText(_translate("Dialog", "请选择压缩方式："))

        #操作选项设计
        self.pushButton= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton.setObjectName('pushButton')
        self.pushButton.setText(_translate("Dialog", 'huffman压缩'))
        self.pushButton.clicked.connect(self.HuffmanCompress)
        self.verticalLayout.addWidget(self.pushButton)
        self.pushButton_2= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_2.setObjectName('pushButton_2')
        self.pushButton_2.setText(_translate("Dialog", 'LZMA压缩'))
        self.pushButton_2.clicked.connect(self.LZMACompress)
        self.verticalLayout.addWidget(self.pushButton_2)
        self.pushButton_3= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_3.setObjectName('pushButton_3')
        self.pushButton_3.setText(_translate("Dialog", '不压缩'))
        self.pushButton_3.clicked.connect(self.Encrypt)
        self.verticalLayout.addWidget(self.pushButton_3)

        #取消按钮设计，取消当前操作，返回初始界面
        self.cancel=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.cancel.setObjectName('cancel')
        self.cancel.setText(_translate('Dialog','取消'))
        self.cancel.clicked.connect(self.reset)
        self.verticalLayout.addWidget(self.cancel)

    #霍夫曼解压，将目标文件按霍夫曼编码方式解压，并去除前缀名
    def HuffmanDecompress(self,file):
        try:
            huffmanDecompress(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{file}无法解压',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解压至{self.path+file[8:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #LZMA解压，将目标文件按LZMA方式解压，并去除前缀名
    def LZMADecompress(self,file):
        try:
            lzmaDecompress(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{file}无法解压',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解压至{self.path+file[5:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #解压操作界面，根据文件前缀名识别其压缩方式并自动执行相应解压操作
    def Decompress(self):
        self.clear()
        new_targets=[]
        for file in self.targets:
            if file[:8]=='huffman_':
                self.HuffmanDecompress(file)
                new_targets.append(file[8:])
            elif file[:5]=='LZMA_':
                self.LZMADecompress(file)
                new_targets.append(file[5:])
            else:
                new_targets.append(file)
        self.targets=new_targets
        self.Unpack()

    #3DES压缩，将目标文件以3DES方式加密，并添加相应前缀名
    def Encrypt3DES(self):
        try:
            #生成密钥
            key=randomString(16)
            
            encrypt3DES(self.dst_path,self.file,key)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法加密',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件加密至{self.dst_path}3DES_{key}_{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.reset()
    
    #AES压缩，将目标文件以AES方式压缩，并添加相应前缀名
    def EncryptAES(self):
        try:
            #生成密钥
            key=randomString(16)
            
            encryptAES(self.dst_path,self.file,key)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法加密',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件加密至{self.dst_path}AES_{key}_{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.reset()

    #压缩操作界面，选择压缩方式
    def Encrypt(self):
        self.clear()
        _translate = QtCore.QCoreApplication.translate

        #提示信息更新
        self.label.setText(_translate("Dialog", "请选择加密方式："))

        #操作选项设计
        self.pushButton= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton.setObjectName('pushButton')
        self.pushButton.setText(_translate("Dialog", '3DES加密（仅支持处理文本文件）'))
        self.pushButton.clicked.connect(self.Encrypt3DES)
        self.verticalLayout.addWidget(self.pushButton)
        self.pushButton_2= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_2.setObjectName('pushButton_2')
        self.pushButton_2.setText(_translate("Dialog", 'AES加密'))
        self.pushButton_2.clicked.connect(self.EncryptAES)
        self.verticalLayout.addWidget(self.pushButton_2)
        self.pushButton_3= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_3.setObjectName('pushButton_3')
        self.pushButton_3.setText(_translate("Dialog", '不加密'))
        self.pushButton_3.clicked.connect(self.reset)
        self.verticalLayout.addWidget(self.pushButton_3)

        #取消按钮设计，取消当前操作，返回初始界面
        self.cancel=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.cancel.setObjectName('cancel')
        self.cancel.setText(_translate('Dialog','取消'))
        self.cancel.clicked.connect(self.reset)
        self.verticalLayout.addWidget(self.cancel)

    #3DES解密，将目标文件按3DES方式解密，并去除前缀名
    def Decrypt3DES(self,file):
        try:
            decrypt3DES(self.path,file,self.dst_path)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解密',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解密至{self.path+file[5:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #AES解密，将目标文件按AES方式解密，并去除前缀名
    def DecryptAES(self,file):
        try:
            decryptAES(self.path,file,self.dst_path)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解密',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解密至{self.path+file[4:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #解密操作界面，根据文件前缀名识别其加密方式并自动执行相应解密操作
    def DecryptFile(self):
        self.clear()
        if not self.targets:
            QtWidgets.QMessageBox.critical(None,'操作失败','请选择至少一个文件进行还原',QtWidgets.QMessageBox.StandardButton.Ok)
            self.setupUI(self.Dialog)
        else:
            new_targets=[]
            for file in self.targets:
                if file[:5]=='3DES_':
                    self.Decrypt3DES(file)
                    new_targets.append(file[22:])
                elif file[:4]=='AES_':
                    self.DecryptAES(file)
                    new_targets.append(file[21:])
                else:
                    new_targets.append(file)
            self.targets=new_targets
            self.Decompress()

    #还原操作界面，选择目标路径下需要还原的文件，文件须为zip或tgz类型
    def Decrypt(self):
        self.clear()
        self.files=[file for file in os.listdir(self.path) if file[-4:]=='.zip' or file[-4:]=='.tgz']
        self.targets=[]

        #提示信息更新
        _translate = QtCore.QCoreApplication.translate
        self.label.setText(_translate("Dialog", "请选择需要还原的文件："))

        #文件选项设计，为目标路径下每个类型为zip或tgz的文件创建一个多选框类型的按钮
        self.showFiles(0)

    
    #打开设置对话框
    def openSettings(self):
        settings_dialog = SettingsDialog(self.Dialog, self.config_manager, self)
        settings_dialog.exec()
    
    #退出应用程序
    def exitApplication(self):
        #创建确认对话框
        reply = QtWidgets.QMessageBox.question(
            self.Dialog,
            '确认退出',
            '您确定要退出智能文件管理系统吗？',
            QtWidgets.QMessageBox.StandardButton.Yes | QtWidgets.QMessageBox.StandardButton.No,
            QtWidgets.QMessageBox.StandardButton.No
        )
        
        #确认退出，关闭窗口
        if reply == QtWidgets.QMessageBox.StandardButton.Yes:
            #停止备份线程
            if hasattr(self, 'backup_local_thread') and self.backup_local_thread:
                self.backup_local_thread.stop()
            if hasattr(self, 'backup_online_thread') and self.backup_online_thread:
                self.backup_online_thread.stop()
        
            # 保存配置
            if hasattr(self, 'config_manager'):
                self.config_manager.save_config()
            self.Dialog.close()
            #QtWidgets.QApplication.quit()
    
    #显示关于对话框
    def showAbout(self):
        about_text = """
        <h3>智能文件管理系统 v1.0</h3>
        <p>一个集成了文件打包、压缩、加密和备份功能的综合管理系统。</p>
        <p><b>主要功能：</b></p>
        <ul>
            <li>文件备份与还原</li>
            <li>多种打包格式（zip、tgz）</li>
            <li>多种压缩算法（Huffman、LZMA）</li>
            <li>多种加密算法（3DES、AES）</li>
            <li>自动本地备份</li>
            <li>自动云备份（GitHub）</li>
        </ul>
        """
    
        QtWidgets.QMessageBox.about(self.Dialog, "关于智能文件管理系统", about_text)
    
    #主目录组件再命名，
    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "智能文件管理系统"))
        self.label.setText(_translate("Dialog", "欢迎使用智能文件管理系统"))
        self.pushButton.setText(_translate("Dialog", "文件备份"))
        self.pushButton_2.setText(_translate("Dialog", "文件还原"))
        self.pushButton_3.setText(_translate("Dialog", "系统设置"))
        self.pushButton_4.setText(_translate("Dialog", "退出系统"))
        self.pushButton_5.setText(_translate("Dialog", "关于"))

#本地备份线程，在程序运行时自动定期将项目文件保存到本地
class backupLocalThread(Thread):
    def __init__(self,interval,path):
        super().__init__()
        self.interval=interval
        self.event=Event()
        self.path=path
        self.daemon=True

    def run(self):
        while not self.event.is_set():
            backupLocal(self.path)
            self.event.wait(self.interval)

    def stop(self):
        self.event.set()

#云备份线程，在程序运行时自动定期将项目文件推送到GitHub
class backupOnlineThread(Thread):
    def __init__(self,interval):
        super().__init__()
        self.interval=interval
        self.event=Event()
        self.daemon=True

    def run(self):
        while not self.event.is_set():
            backupGitHub()
            self.event.wait(self.interval)

    def stop(self):
        self.event.set()

#构建集成了GUI界面与备份线程的主类
class MyMainForm(QtWidgets.QMainWindow,Ui_Dialog):
    def __init__(self):
        super().__init__() #parent=None
        self.init(self)
        self.initBackupThreads()

    #备份线程初始化，为本地及云备份线程设定间隔时间并将其启动
    def initBackupThreads(self):
        #从配置读取间隔时间
        local_interval = self.config_manager.get("backup", "local_interval")
        cloud_interval = self.config_manager.get("backup", "cloud_interval")
        
        #检查是否启用备份
        backup_enabled = self.config_manager.get("system", "backup_enabled")
        cloud_enabled = self.config_manager.get("system", "cloud_backup_enabled")


        #判断是否进行本地备份
        if backup_enabled:
            path = self.config_manager.get("backup", "backup_dir")
            self.backup_local_thread = backupLocalThread(local_interval, path)
            self.backup_local_thread.start()
        else:
            self.backup_local_thread = None

        #判断是否进行云备份
        if cloud_enabled:
            self.backup_online_thread = backupOnlineThread(cloud_interval)
            self.backup_online_thread.start()
        else:
            self.backup_online_thread = None
        
        '''self.backup_local_thread=backupLocalThread(10000)
        self.backup_online_thread=backupOnlineThread(10000)
        self.backup_local_thread.start()
        self.backup_online_thread.start()'''

    #更新备份线程设置
    def updateBackupThreads(self):
        # 停止现有线程
        if self.backup_local_thread:
            self.backup_local_thread.stop()
        if self.backup_online_thread:
            self.backup_online_thread.stop()
        
        # 重新初始化线程
        self.initBackupThreads()

#主函数
if __name__=='__main__':
    app=QtWidgets.QApplication(sys.argv)
    window=MyMainForm()
    window.show()
    sys.exit(app.exec())



No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit
No changes to commit


100%|██████████| 3949688/3949688 [00:02<00:00, 1785716.14byte/s]


No changes to commit
No changes to commit
